In [ ]:
import os
import pickle
import sys
sys.path.append("../")

import importlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics

from openbustools import plotting
from openbustools.traveltime import model_utils

In [ ]:
all_res = []
for model_res_file in os.listdir("../results/kcm/"):
    res = model_utils.format_model_res(f"../results/kcm/{model_res_file}")
    all_res.append(res)
all_res = pd.concat(all_res)
all_res['model_archetype'] = all_res['model'].str.split('_').str[0]
all_res['is_tuned'] = False
all_res.loc[all_res['model'].str.split('_').str[-1]=='TUNED', 'is_tuned'] = True
all_res

In [ ]:
subset = all_res[all_res['metric']=='mae']
subset = subset[subset['is_tuned']==False]
subset = subset[subset['model_archetype'].isin(['GRU','AVGM','AVGH','PERT'])]
subset = subset.sort_values('model')
sns.barplot(subset, x='value', y='model', hue='experiment_name')

In [ ]:
run_folder_kcm = "../results/big_run/kcm/models/"
run_folder_atb = "../results/big_run/atb/models/"
run_folder_mix = "../results/big_run_nosch/kcm_atb/models/"
run_folder_rut = "../results/big_run_nosch/rut/models/"

### Model Performance

In [ ]:
print(kcm_gen_df[kcm_gen_df['Loss']=="Train_Losses"].groupby(["Model"])[["MAPE"]].agg(["mean","std"]))

In [ ]:
print(kcm_result_df.groupby(["Model"])[["MAPE"]].agg(["mean","std"]))